---
title: "SCADA-Based Reliability Evaluation"
subtitle: "Kelmarsh Wind Farm - 12.3 MW"
author: "Jorge A. Thomas"
date: 23.April.2025
format:    
    html:
      theme: default
      self-contained: true
      code-fold: true
      df-print: tibble
      code-summary: "Show the code"
      toc: true
      toc-depth: 2
      grid: 
        margin-width: 350px      
execute: 
  echo: false
reference-location: margin 
citation-location: document
bibliography: references.bib
nocite: '@*'
---


## Kelmarsh Wind Farm

### Project Overview

"Kelmarsh comprises six 2.05MW Senvion MM92 turbines. The Project was acquired from EON in December 2014 and was constructed in joint venture with Santander using balance sheet finance. Construction was completed in April 2016. It is now owned by Cubico Investments."

- [https://www.blue-energyco.com/our-projects/kelmarsh/](https://www.blue-energyco.com/our-projects/kelmarsh/)  

::: {#fig-intro layout-ncol="2"}
![(a) Kelmarsh Wind Farm](./online_assets/Kelmarsh_wind_project_blue-energyco.jpg){width="98%"}

![(b) Senvion MM92 turbine](./online_assets/turbine-senvion_mm92.jpg){width="98%"}

Kelmarsh Wind Farm is located near Haselbach, Northamptonshire, UK.
:::


In [ ]:
#| label: Pyhton-setup
#| eval: true
#| echo: false

# https://en.wind-turbine-models.com/turbines/889-senvion-mm92
# https://www.thewindpower.net/windfarm_en_17507_kelmarsh.php

# Plumley, C. (2022). Penmanshiel wind farm data (0.1.0) [Kelmarsh dataset]
# Cubico Sustainable Investments

from pathlib import Path
import duckdb
import polars as pl
import polars.selectors as cs
import plotly.express as px
from great_tables import GT, md, style, loc

fpath_processed = "../data/processed/"
fpath_interim = "../data/interim/"
fpath_online = "./online_assets/"

In [ ]:
#| label: Read-Data
#| eval: true

tstatus = pl.read_parquet(f"{fpath_online}Kelmarsh_Turbines_Status_20160114_20230109_n385133_cols12.parquet")

## Events Catalogue and Status


In [ ]:
#| label: Events-Catalogue
#| eval: true
#| warning: true

events_catalogue = (
    tstatus.group_by("IEC category", "Status")
    .agg(pl.len().alias("Count"))    
    .pivot(
        values="Count",
        index="IEC category",
        on="Status"
    )
    .fill_null(0)
)

GT(events_catalogue).tab_source_note(source_note='Note: Only "Forced outage" events with status "Stop" are considered for the evaluation, i.e., 892 events.')

In [ ]:
#| label: Filter-Stops
#| eval: true

null_stop_events = pl.sql(    
    """
    SELECT * EXCLUDE ("Custom contract category")    
    FROM tstatus
    WHERE "IEC category" IS NULL AND "Status" = 'Stop'

    """).collect()

# null_stop_events['Status'].unique()

curtailment = pl.sql(    
    """
    SELECT * EXCLUDE ("Custom contract category")    
    FROM tstatus
    WHERE "Status" = 'Curtailment'
    """).collect()


stops = pl.sql(    
    """
    SELECT * EXCLUDE ("Custom contract category")    
    FROM tstatus
    WHERE "Status" = 'Stop'
    """).collect()

# stops.group_by('IEC category').agg(pl.len().alias("Stop Events"))

In [ ]:
#| label: Filter-Forced-Outages
#| eval: true

# Aggregate tables using SQL syntax

foutages = pl.sql(
    """
    SELECT * 
    FROM stops
    WHERE "IEC category" = 'Forced outage' 
    """).collect()

foutages = foutages.rename({"Duration":"TTR"})

# TTF
foutages = foutages.with_columns(
    (pl.col("Timestamp start").shift(-1) - pl.col("Timestamp end")).shift(1).alias("TTF")
    )

foutages = foutages.with_columns(
    pl.when(pl.col("TTF") < 0).then(None).otherwise(pl.col("TTF")).alias("TTF")
)

In [ ]:
#| label: write-foutages-online
#| eval: false

# foutages.write_parquet(f"{fpath_online}/{"Kelmarsh_Turbines_ForcedOutages_20160114_20230109_n892_cols12.parquet"}")

In [ ]:
#| label: read-filtered-fo
#| eval: false

# foutages = pl.read_parquet("./online_assets/Kelmarsh_Turbines_ForcedOutages_20160114_20230109_n892_cols12.parquet")

# foutages

In [ ]:
#| label: Binary-Plot
#| eval: true

# Generate data frame with 0-1 states

dtstarts = foutages.select(["Timestamp start", "Status", "Message",  "wt"]).rename({"Timestamp start": "Timestamp"})

dtends = foutages.select(["Timestamp end", "Status", "Message",  "wt"]).with_columns(pl.lit("Start").alias("Status")).rename({"Timestamp end": "Timestamp"})

# Vertical concatenation
tstates = pl.concat([dtstarts, dtends]).sort(["wt", 'Timestamp'])

tstates= tstates.with_columns(
    pl.when(pl.col("Status") == "Stop").then(0).otherwise(1).alias("State"),
    pl.col("Timestamp").dt.year().alias("Year")    
    )

# Extract last year 
last_year = tstates["Year"].unique().max()

tstates= tstates.sort(["Year", "wt", "Timestamp"]).drop_nulls()

In [ ]:
#| label: e-o-y-algo

# TTF (or TTR) censoring  algorithm per Turbine per year.

# ---- 1) Id  last row per year per wt

last_rows = tstates.group_by(["Year", "wt"]).last().sort("Year")

# ---- 2) Create new rows with the same (last seen) State

# This closes the year apporaching from the left 

last_rows = last_rows.with_columns(    
    pl.col("Timestamp").dt.truncate("1y").dt.offset_by("1y").dt.offset_by("-1s").alias("Timestamp"),  # Ceil timestamp
    pl.lit("Last-seen-Censored").alias("Message")
    )

# ---- 3) Create first row df from last_rows df

first_rows = last_rows.with_columns(
     pl.col("Timestamp").dt.offset_by('1s').alias("Timestamp")
)

# ---- 4) df with new rows to be added

new_censored_rows = pl.concat([first_rows, last_rows])

# Add Year columns
new_censored_rows = new_censored_rows.with_columns(
    pl.col("Timestamp").dt.year().alias("Year")    
)

# Recompute Year_wt
# new_censored_rows = new_censored_rows.with_columns(   
#     (pl.col("Year").cast(str) + "-" + pl.col("wt")).alias("Year_wt")
# )

# ---- 5) Add new rows to close tstates

new_censored_rows = new_censored_rows.select(tstates.columns)  

# reorder rows
tstates = pl.concat([tstates, new_censored_rows]).sort(["wt", "Timestamp"])

# ---- 6) Drop columns Year > 2022  
tstates = tstates.filter(pl.col("Year") <= last_year)

In [ ]:
#| label: Find-Missing-Values
#| eval: false

# Get rows containing any null values
missing = (tstates
          .with_row_count("row_index")
          .filter(pl.any_horizontal(pl.all().is_null()))
          .sort("row_index"))

# Display rows with missing values
print("Rows containing missing values:")
print(missing)

# Get count of missing values per column
print("\nMissing value counts per column:")
print(tstates.null_count())

In [ ]:
#| label: Square-Signal-Plot
#| eval: false

# Other line_shape options, or interpolation methods between given points:

# 'hv' step ends, equivalent to pyplot's post option;
# 'vh' step starts;
# 'hvh' step middles, x axis;
# 'vhv' step middles, y axis;
# 'spline' smooth curve between points;
# 'linear' line segments between points, default value for line_shape.

fig = px.line(tstates,
              x='Timestamp', 
              y="State", 
              line_shape='hv', 
              facet_row="Year_wt",
              color="wt",
              hover_data=["Timestamp", "Message", "Status"],  # Add the variables you want in tooltip
              title="Time / State Diagram for Kellmarsh Wind Turbines (2016-2022)",
              subtitle= "Step-down = Forced outage (IEC); Step-up=Start",
              labels={"Timestamp": "UTC", "State": "", "wt": ""}, 
              facet_row_spacing=0.01
              )
# Adjust spacing between facets
fig.update_layout(
    height=1200,  # Increase overall height
    width=800,   # Set overall width
    margin=dict(l=100, r=50, t=100, b=50),  # Adjust margins
    showlegend=False,
    legend=dict(orientation="h",
                yanchor="bottom",
                y=1.01,
                xanchor="right",
                x=1,
                font=dict(size=8),  # Reduce legend font size
                itemsizing='constant',  # Make legend items more compact
                itemwidth=30  # Reduce width of legend items
                )                
)

# Update y-axis to show only 0 and 1 as labels
fig.update_yaxes(
    tickvals=[0, 1],  # Set tick values to 0 and 1
    ticktext=["", ""]  # Optional: Explicitly set tick text
)

fig.update_xaxes(matches=None, 
tickformat="%b",
showgrid=True
)

# Update facet labels orientation and size
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1], textangle=0, xanchor='left', x=-0.08))  # labels horizontal
fig.update_annotations(font_size=9)  # Reduce font size

fig.show()

In [ ]:
#| label: Time-State-Plot-By-Year

# Get unique years
years = tstates["Year"].unique().sort()

# Create a plot for each year
pl_time_state = []
for year in years:
    # Filter data for current year
    year_data = tstates.filter(pl.col("Year") == year)
    
    fig = px.line(year_data,
                  x='Timestamp', 
                  y="State", 
                  line_shape='hv', 
                  facet_row="wt",
                  color="wt",
                  hover_data=["Message", "Status"],
                  title="Step-down: Forced outage (IEC) | Step-up: Start",
                  #subtitle= "(0) Down Step=Forced outage (IEC), (1) Up Step=Start",
                  labels={"Timestamp": "UTC", "State": "", "wt": ""}, 
                  facet_row_spacing=0.01
                  )
    # Update layout
    fig.update_layout(
        height=700,
        width=700,
        margin=dict(l=100, r=50, t=100, b=50),
        showlegend=False,
        legend=dict(orientation="h",
                    yanchor="bottom",
                    y=1.01,
                    xanchor="right",
                    x=1,
                    font=dict(size=8),
                    itemsizing='constant',
                    itemwidth=30
                    )                
    )
    # Update axes
    fig.update_yaxes(
        tickvals=[0, 1],
        ticktext=["", ""]
    )
    fig.update_xaxes(
        matches='x',
        tickformat="%b",
        dtick="M1",  # Show ticks every month
        showgrid=True
    )
    # Update facet labels
    fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1], 
                                               textangle=0, 
                                               xanchor='left', 
                                               x=-0.05))
    fig.update_annotations(font_size=10)
    pl_time_state.append(fig)

# End of loop

In [ ]:
#| label: calc-durations
#| eval: true

tstates_dur = tstates.with_columns(
    (pl.col("Timestamp").diff().dt.total_seconds()/3_600).alias("Duration").shift(-1)
)

# CENSOR data around years transitions

tstates_dur = tstates_dur.with_columns(
    pl.when((pl.col("Duration") <= 0) | (pl.col("Message") =="Last-seen-Censored"))
    .then(None)
    .otherwise(pl.col("Duration"))
    .alias("Duration")    
)

tstates_dur =  tstates_dur.drop_nulls()

# # Get each year duration
# year_durations = tstates_dur.group_by("Year").agg(
#     pl.col("Timestamp").max().alias("year_end"),
#     pl.col("Timestamp").min().alias("year_start")
# ).with_columns(
#     (pl.col("year_end") - pl.col("year_start")).dt.total_seconds().alias("duration_sec")
# ).sort("Year")

# year_durations = year_durations.with_columns(
#     (pl.col("duration_sec")/3_600).round(2).alias("duration_hours")
# ).drop("duration_sec")

tstates_dur = tstates_dur.rename({"wt": "Turbine"}).drop_nulls()
tstates_dur = tstates_dur.sort(["Turbine", "Timestamp"])

In [ ]:
#| label: calc-mttf-mttr

ttf = tstates_dur.filter(pl.col("State") == 1).rename({"Duration": "TTF"})

ttf_agg_year = (    
    ttf.group_by(["Year",  "Turbine"])
    .agg(
        pl.len().alias("State delete"),                
        pl.sum("TTF").alias("Time to Failure"),
        pl.mean("TTF").alias("MTTF")               
        )         
    .sort(["Year",  "Turbine"])     
    )

ttr = tstates_dur.filter(pl.col("State") == 0).rename({"Duration": "TTR"})

ttr_agg_year = (    
    ttr.group_by(["Year",  "Turbine"])
    .agg(
        pl.len().alias("Forced Outages"),                
        pl.sum("TTR").alias("TTR"),
        pl.mean("TTR").alias("MTTR")                      
        )    
    .sort(["Year",  "Turbine"])      
    )

In [ ]:
#| label: save-ttf-ttr
#| eval: false

# metadata for filename

first_ts = ttr["Year"].min()
last_ts = ttr["Year"].max()

# Additional metadata options
# null_pct = int(rel_param["MTTF"].null_count() / len(silistea) * 100)

filename_ttf = f"Kelmarsh_time-to-failure_{first_ts}_{last_ts}_n{len(ttf)}_cols{len(ttf.columns)}.parquet"

# Save processed dataset
ttf.write_parquet(f"{fpath_processed}{filename_ttf}")

filename_ttr = f"Kelmarsh_time-to-repair_{first_ts}_{last_ts}_n{len(ttr)}_cols{len(ttr.columns)}.parquet"

# Save processed dataset
ttr.write_parquet(f"{fpath_processed}{filename_ttr}")

In [ ]:
#| label: calc-outages-month
#| eval: true

monthly_outages = ttr.with_columns(
    pl.col("Timestamp").dt.month().alias("Month")
)

monthly_outages = (
    monthly_outages.group_by(["Year", "Turbine", "Month"])
    .agg(pl.len().alias("Outages"))
    .sort(["Year", "Turbine", "Month"])
    )

monthly_outages_piv = monthly_outages.pivot(
        values="Outages",
        index=["Year", "Turbine"],
        on="Month",
        aggregate_function="first"
    ).fill_null(strategy="zero")

monthly_outages_piv = monthly_outages_piv.select(
    ["Year", "Turbine", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12"]).sort("Year", "Turbine")

monthly_outages_ = monthly_outages_piv.with_columns(
    pl.concat_list(pl.all().exclude("Year", "Turbine")).alias("Monthly")
    ).select(["Year", "Turbine", "Monthly"])

In [ ]:
#| label: calc-reliablity-params
#| eval: true

rel_param = ttf_agg_year.join(ttr_agg_year, on=["Year", "Turbine"], how="left")
rel_param = rel_param.join(monthly_outages_, on=["Year", "Turbine"])
rel_param = rel_param.drop("State delete")
rel_param = rel_param.rename({"TTR":"Downtime", "Time to Failure": "Uptime"})

rel_param = rel_param.with_columns(
    (pl.col("Uptime") + pl.col("Downtime")).alias("EvalTime")
)

rel_param = rel_param.with_columns(
    (pl.col("Forced Outages")/pl.col("EvalTime")).alias("Failure Rate"),
    (pl.col("MTTR")/(pl.col("MTTF") + pl.col("MTTR"))).alias("FOR"),    
)

rel_param = rel_param.with_columns(    
    (1-pl.col("FOR")).alias("Availability"),
    pl.lit("wind-turbine-icon-10.jpg").alias("icon")
)

In [ ]:
#| label: write-rel-eval-results
#| eval: false

# metadata for filename

first_ts = rel_param["Year"].min()
last_ts = rel_param["Year"].max()

# Additional metadata options
# null_pct = int(rel_param["MTTF"].null_count() / len(silistea) * 100)

filename = f"Kelmarsh_reliability_paramaters_{first_ts}_{last_ts}_n{len(rel_param)}_cols{len(rel_param.columns)}.parquet"

# Save processed dataset
rel_param.write_parquet(f"{fpath_processed}{filename}")

In [ ]:
#| label: generate-table
#| eval: true

gt_reliability = (
    GT(rel_param,
    rowname_col = "Turbine",
    groupname_col="Year"
    )
    .opt_vertical_padding(scale=0.6)
    .opt_horizontal_padding(scale=1)
    .opt_stylize(style=1)
    .tab_style(
        style=style.text(weight="bold"), 
        locations=loc.body(columns="Year")
    )
    .tab_header(
        title=md("Six Wind Turbines - Model Senvion MM92"),
        subtitle=md("Reliability Evaluation (2016 - 2022)")
    )
    .tab_source_note(source_note="* Forced Outages (IEC) only. Yearly evaluation period -> Censored uptime (TTF) data around year change.")    
    .fmt_number(
        columns=["MTTF", "MTTR", "Downtime", "Uptime", "EvalTime"],
        decimals=2
    )
    .fmt_image(
        columns="icon",
        path="./online_assets/"
    )
    .fmt_number(
        columns=["Failure Rate", "FOR", "Availability"],
        decimals=5
    )
    .fmt_nanoplot(
        columns="Monthly",
        plot_type="bar"
    )
    .tab_spanner(
        label="Probabilities",
        columns=["FOR", "Availability"]
    )
    .tab_spanner(
        label="Out-of-Service [h]",
        columns=["Downtime", "MTTR"]
    )
    .tab_spanner(
        label="*Ready [h]",
        columns=["Uptime", "MTTF"]
    )
    .tab_spanner(
        label="Forced Outages",
        columns=["Forced Outages", "Monthly", "Failure Rate"]
    )
    .cols_move_to_start(
        columns=["Year", "icon", "Turbine", "Forced Outages", "Monthly", "Failure Rate", "Uptime",  "MTTF"]
    )
    .cols_label(        
        {"Forced Outages": "Total",         
        "Failure Rate": "λ [h⁻¹]",               
        "icon": "Turbine",
        "FOR": "FOR ({{P_0}})",
        "Availability": "A ({{P_1}})"
        }
    ).cols_hide(columns="EvalTime")
)

## Reliability Parameters


In [ ]:
#| label: print-great-table

gt_reliability

## Time / State Diagrams

::: {.panel-tabset}

## 2016


In [ ]:
pl_time_state[0].show()

## 2017


In [ ]:
pl_time_state[1].show()

## 2018


In [ ]:
pl_time_state[2].show()

## 2019


In [ ]:
pl_time_state[3].show()

## 2020


In [ ]:
pl_time_state[4].show()

## 2021


In [ ]:
pl_time_state[5].show()

## 2022


In [ ]:
pl_time_state[6].show()

:::

## Histogram of Failure Causes

### Total


In [ ]:
#| label: failure-hist
#| eval: true

import plotly.express as px

# Count the frequency of each Message
message_counts = ttr.group_by("Message").agg(
    pl.len().alias("Count")
).sort("Count", descending=True)

# Create the horizontal bar plot
fig = px.bar(
    message_counts,
    x="Count",
    y="Message",
    hover_data="Count",
    orientation='h',  # Makes the bars horizontal    
    labels={"Message": "", "Count": "Frequency"},
)

# Update layout for better readability
fig.update_layout(
    height=1000,  # Increased height to accommodate all labels
    width=770,
    yaxis={'categoryorder':'total ascending',
           'tickangle': 0  # Make y-axis labels horizontal
    },  # Sort bars by count
    bargap=0.3 
)
fig.update_annotations(font_size=8)
# fig.update_traces(textposition='outside')
# fig.for_each_annotation(textangle=0)  # labels horizontal

fig.show()

### Per Wind Turbine


In [ ]:
#| label: histrogram-per-turbine
#| eval: true

# Count the frequency of each Message per Turbine
message_counts_per_wt = ttr.group_by(["Turbine", "Message"]).agg(
    pl.len().alias("Count")
).sort(["Turbine", "Count"], descending=[False, True])

# Create the horizontal bar plot with facets
fig_2 = px.bar(
    message_counts_per_wt,
    x="Count",
    y="Message",
    hover_data="Count",
    facet_col="Turbine",  # Create separate plots per turbine
    orientation='h',    
    labels={"Message": "", "Count": "Freq.", "Turbine": ""},
)

# Update layout for better readability
fig_2.update_layout(
    height=1000,
    width=770,  # Increased width to accommodate facets
    yaxis={'categoryorder':'total ascending',
           'tickangle': 0  # Make y-axis labels horizontal
    },
    bargap=0.2,
    showlegend=False
)
fig.update_annotations(font_size=8)
# Update facet settings
fig_2.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))  # Remove "Turbine=" from facet titles

fig_2.show()